Getting Pulses Using `pyRawIO` 
==============================

The SuperCDMS library `pyRawIO` allows us to read raw MIDAS DAQ files directly to get the raw pulses that our analysis RQs are based on. The library is hosted on the GitLab page for SuperCDMS [[GitLab:pyRawIO][GitLab:pyRawIO]]. The library `pyRawIO` depends on other libraries like `CDMSbats` and `ROOT`-- those are sometimes difficult to install so these instructions focus on using a `singularity` container [[Singularity][Singularity]] where all of that stuff is already pre-installed and all you need to run this is to install `singularity` and have the image files (which are maintained by SuperCDMS [[Singularity-Images][Singularity-Images]]). 

[GitLab:pyRawIO]: https://gitlab.com/supercdms/DataHandling/pyRawIO "GitLab:pyRawIO"
[Singularity]: https://docs.sylabs.io/guides/latest/user-guide/introduction.html "Singularity" 
[Singularity-Images]: https://confluence.slac.stanford.edu/display/CDMS/Using+Containerized+CDMS+Offline+Releases#UsingContainerizedCDMSOfflineReleases-WheretogettheSingularityimagesfrom? "SCDMS Singularity Image Registry"

Singularity
-----------

To begin using singularity you will probably have to install it, but test if you have it by typing `singularity --help` in the terminal; if that returns the help menu then you already have it! Singularity depends on go [[go][go]], so if you don't have it you will have to install that first. Install `singularity` and `go` according to the instructions in this repository [[example_scripts/singularity][sing-install]].

Here are some useful singularity images on disk on the UMN cluster:

 - `/data/chocula/villaa/cdmsfull_latest.sif` (don't remember exactly the SuperCDMS release tag, V04-12?)
 - `/data/chocula/villaa/cdmsfull_V04-07-00.sif`  
 - `/data/chocula/villaa/cdmsfull_V04-13-00.sif`
 
Before using `singularity` you should have the correct environmental variables set, this could be:

```
export GOCACHE="/data/chocula/villaa/.cache/go"
export GOMODCACHE="/data/chocula/villaa/.cache/go/pkg/mod"
export GOPATH="/data/chocula/villaa/go"
export CCACHE_DIR="/data/chocula/villaa/.ccache"
export PATH=$PATH:/data/chocula/villaa/goinstall/go/bin
export SINGULARITY_BINDPATH=/data
```

Most of these set up the paths and cache directories for the language `go` itself. Make the cache directories something that has some space. On the UMN cluster my home directory is severely limited so I used the data directory `/data/chocula/villaa`, other users should be able to make their own directory in `/data/chocula` if they don't already have one. Contact Anthony Villano to get help with setting that up if you don't have permission. 

Open a shell in `singularity` then you will have access to the code environments above created for SuperCDMS and having `pyRawIO`, `CDMSbats` and `ROOT` already installed.

```
$ singularity shell /data/chocula/villaa/cdmsfull_V04-13-00.sif 
WARNING: DEPRECATED USAGE: Environment variable SINGULARITY_BINDPATH will not be supported in the future, use APPTAINER_BINDPATH instead
Singularity> 
```

[go]: https://go.dev/doc/install "GO programming language"
[sing-install]: https://github.com/villano-lab/R76Analysis/tree/master/example_scripts/singularity "Installing Singularity"

Loading In Events
-----------------

`singularity_pulseSave.py`
-------------------------

Reading Back Events
-------------------